# Labeling Evaluation

This notebook is used to evaluate the manual labels of segmentation results. 
For some experimental manual labeling for smaller clusters like 16, this would help to determine wether a good decision is made and help to adjust the thresholds.

Also used for evaluating the results given by larger clusters labelling by CNNs. 

Workflow:

Labels: draw the four dragongly classes for target slice together to form a multi-class label.

Preduction: draw the kmeans/gmm clusters that are in the same predicted class, coloring as same color. Add them together and compare with labels with macro-averaged metrics.

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import glob
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support


Two ways of evaluation:

1. Treat the problem as multi-class classification, use macro averaged score to evaluate them all together (micro gives accuracy): average='macro', labels=[1,2,3,4], exclude class 0 since we only interested in 4

2. Treat the problem still as multiple binary classification for two reasons: average=None, labels=[1,2,3,4]
   
   a. The small cluster (3-10) experiments still used binary metrics, need to compare with them

   b. There are unclassified pixels in dragonfly labels, which could affect our results, yielding lower scores. 

## As multi-class classification

In [2]:
# dragonfly
def get_dragonfly_label_by_slice(slice_num):
    current_path = os.getcwd()
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite)
    celestite = cv2.imread(dragonfly_celestite)
    gypsum = cv2.imread(dragonfly_gypsum)
    pore = cv2.imread(dragonfly_pore)

    # transfer the reference into 0/1 labels and then turn to its class number
    # 0: other;
    # 1: pore;
    # 2: gypsum;
    # 3: celestite;
    # 4: bassanite
    pore_label = (pore == [255,255,255]).all(axis=2) * 1
    gypsum_label = (gypsum == [255,255,255]).all(axis=2) * 2
    celestite_label = (celestite == [255,255,255]).all(axis=2) * 3
    bassanite_label = (bassanite == [255,255,255]).all(axis=2) * 4

    label = pore_label + gypsum_label + celestite_label + bassanite_label

    return label.flatten()

In [3]:
# gmm / k-means
def get_unsupervised_pred_by_slice(manual_label, seg_model, seg_nd, cluster_num, slice_num):
    current_path = os.getcwd()
    seg_path = os.path.join(current_path, 'new_large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))
    seg_res_list = glob.glob(os.path.join(seg_path, str(slice_num), '*.png'))
    
    img = 0

    assert len(manual_label) == len(seg_res_list) == cluster_num

    for seg in seg_res_list:
        # our segmentation results has invered color, black [0,0,0] is the interested class
        c = int(os.path.basename(seg[-16:-13]))
        if manual_label[c] == 1:  # pore
            pore = cv2.imread(seg)
            pore = (pore == [0,0,0]).all(axis=2) * 1
            img += pore
        elif manual_label[c] == 2:  #gypsum
            gypsum = cv2.imread(seg)
            gypsum = (gypsum == [0,0,0]).all(axis=2) * 2
            img += gypsum
        elif manual_label[c] == 3:  #celestite
            celestite = cv2.imread(seg)
            celestite = (celestite == [0,0,0]).all(axis=2) * 3
            img += celestite
        elif manual_label[c] == 4:  #bassanite
            bassanite = cv2.imread(seg)
            bassanite = (bassanite == [0,0,0]).all(axis=2) * 4
            img += bassanite

    return img.flatten()


In [4]:
# dragonfly
#return the 4 labels separetely as binary 1d-array
def get_dragonfly_label_by_class(slice_num):
    current_path = os.getcwd()
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite)
    celestite = cv2.imread(dragonfly_celestite)
    gypsum = cv2.imread(dragonfly_gypsum)
    pore = cv2.imread(dragonfly_pore)

    # transfer the reference into 0/1 labels and then turn to its class number
    # 0: other;
    # 1: pore;
    # 2: gypsum;
    # 3: celestite;
    # 4: bassanite
    pore_label = (pore == [255,255,255]).all(axis=2)
    gypsum_label = (gypsum == [255,255,255]).all(axis=2)
    celestite_label = (celestite == [255,255,255]).all(axis=2)
    bassanite_label = (bassanite == [255,255,255]).all(axis=2)

    return pore_label.flatten(), gypsum_label.flatten(), celestite_label.flatten(), bassanite_label.flatten()

In [5]:
# gmm / k-means
# concate pixels of each class together, and return the 4 predictions as binary 1d-array
def get_unsupervised_pred_by_class(manual_label, seg_model, seg_nd, cluster_num, slice_num):
    current_path = os.getcwd()
    seg_path = os.path.join(current_path, 'large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))
    seg_res_list = glob.glob(os.path.join(seg_path, str(slice_num), '*.png'))
    
    img1 = 0
    img2 = 0
    img3 = 0
    img4 = 0

    assert len(manual_label) == len(seg_res_list) == cluster_num

    for seg in seg_res_list:
        # our segmentation results has invered color, black [0,0,0] is the interested class
        c = int(os.path.basename(seg[-16:-13]))
        if manual_label[c] == 1:  # pore
            pore = cv2.imread(seg)
            pore = (pore == [0,0,0]).all(axis=2)
            img1 += pore
        elif manual_label[c] == 2:  #gypsum
            gypsum = cv2.imread(seg)
            gypsum = (gypsum == [0,0,0]).all(axis=2)
            img2 += gypsum
        elif manual_label[c] == 3:  #celestite
            celestite = cv2.imread(seg)
            celestite = (celestite == [0,0,0]).all(axis=2)
            img3 += celestite
        elif manual_label[c] == 4:  #bassanite
            bassanite = cv2.imread(seg)
            bassanite = (bassanite == [0,0,0]).all(axis=2)
            img4 += bassanite

    return img1.flatten(), img2.flatten(), img3.flatten(), img4.flatten()

# Evaluation of Labeling based on all 401 results

In [4]:
from auto_label import auto_label, PRECISION_50, PRECISION_X, FSCORE_SOLO, PRECISION_SOLO

In [7]:
def evaluate_label_per_slice(seg_model, seg_nd, cluster_num, slice_num, mode, threshold):
    # get class labels assigned to each cluster
    class_label = auto_label(seg_model, seg_nd, cluster_num, mode, threshold)
    #class_label = auto_label_last_200(seg_model, seg_nd, cluster_num, mode, threshold)
    
    # get pixel label and pred of the given slice
    pixel_pred = get_unsupervised_pred_by_slice(class_label, seg_model, seg_nd, cluster_num, slice_num)
    pixel_label = get_dragonfly_label_by_slice(slice_num)

    p, r, f1, _= precision_recall_fscore_support(pixel_label, pixel_pred, average=None, labels=[1,2,3,4])
    macro_f1 = f1.mean()

    return p, r, f1, macro_f1


cluster 16: 2m 40~s

cluster 32: 5m 0.6s

In [ ]:
seg_nd = '4d'
seg_model = 'gmm'
cluster_num = 128
mode = PRECISION_50
threshold = 0.5

header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']
f = open('{}_{}_{}_label_mode_{}.csv'.format(seg_model, seg_nd, cluster_num, mode), 'w')
writer = csv.writer(f)
writer.writerow(header)

for s in range(400, 801):
    p, r, f1, macro_f1_4 = evaluate_label_per_slice(seg_model, seg_nd, cluster_num, s, mode, threshold)
    f1_3 = np.array([f1[0], f1[1], f1[3]])
    macro_f1_3 = f1_3.mean()
    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
    writer.writerow(data)

f.close()
    

In [ ]:
seg_nd = ['3d','4d']
seg_model = ['gmm', 'k-means']
cluster_num = [16,32,64,128]
mode = [PRECISION_SOLO, FSCORE_SOLO]
threshold = 0.5

header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']

for model in seg_model:
    for nd in seg_nd:
        for c in cluster_num:
            for m in mode:
                print('{}_{}_{}_label_mode_{}.csv'.format(model, nd, c, m))
                f = open('{}_{}_{}_label_mode_{}.csv'.format(model, nd, c, m), 'w')
                writer = csv.writer(f)
                writer.writerow(header)

                for s in range(400, 801):
                    p, r, f1, macro_f1_4 = evaluate_label_per_slice(model, nd, c, s, m, threshold)
                    f1_3 = np.array([f1[0], f1[1], f1[3]])
                    macro_f1_3 = f1_3.mean()
                    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
                    writer.writerow(data)

                f.close()
                print('Finish!')

In [ ]:
seg_nd = ['3d','4d']
seg_model = ['gmm', 'k-means']
cluster_num = [16,32,64,128]
mode = PRECISION_X
threshold = [0.6,0.7,0.8,0.9]

header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']

for model in seg_model:
    for nd in seg_nd:
        for c in cluster_num:
            for t in threshold:
                print('{}_{}_{}_label_mode_{}_{}.csv'.format(model, nd, c, mode, t))
                f = open('{}_{}_{}_label_mode_{}_{}.csv'.format(model, nd, c, mode, t), 'w')
                writer = csv.writer(f)
                writer.writerow(header)

                for s in range(400, 801):
                    p, r, f1, macro_f1_4 = evaluate_label_per_slice(model, nd, c, s, mode, t)
                    f1_3 = np.array([f1[0], f1[1], f1[3]])
                    macro_f1_3 = f1_3.mean()
                    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
                    writer.writerow(data)

                f.close()
                print('Finish!')

# Evaluation of Labeling based on the last 201 slices

In [5]:
def auto_label_last_201(seg_model, seg_nd, cluster_num, mode, threshold=0.5):
    # cluster_num: the total number of clusters
    assert mode in [1,2,3,4], "Invalid mode: mode should be integer in [1,2,3,4]."
    if mode == 1:
        assert threshold == 0.5, "Mode 1 requires threshold = 0.5."

    csv_file = os.path.join(os.getcwd(), 'seg_res_evaluation_csv', '{}_{}_{}.csv'.format(seg_model, seg_nd, cluster_num))
    df = pd.read_csv(csv_file, usecols=['slice', 'current_cluster','pore_precision', 'pore_f1', 'gypsum_precision', 'gypsum_f1', 'celestite_precision', 'celestite_f1', 'bassanite_precision', 'bassanite_f1'])
    df = df.loc[df['slice'] >= 600]  # only use training set

    label = [0]*cluster_num

    for i in range(cluster_num):
        one_cluster = df.loc[df['current_cluster'] == i]
        stats = one_cluster.mean()
        precisions = [stats[2], stats[4], stats[6], stats[8]]
        fscores = [stats[3], stats[5], stats[7], stats[9]]
        p_max = max(precisions)
        p_idx = np.argmax(precisions)
        f_idx = np.argmax(fscores)
        if mode in [1,2]:
            if p_max <= threshold:
                idx = f_idx
            else:
                idx = p_idx
        elif mode == 3:
            idx = f_idx
        else:
            idx = p_idx
            
        class_num = idx + 1
        label[i] = class_num

    return label

In [6]:
def evaluate_label_per_slice(seg_model, seg_nd, cluster_num, slice_num, class_label):
    # get class labels assigned to each cluster
    
    # get pixel label and pred of the given slice
    pixel_pred = get_unsupervised_pred_by_slice(class_label, seg_model, seg_nd, cluster_num, slice_num)
    pixel_label = get_dragonfly_label_by_slice(slice_num)

    p, r, f1, _= precision_recall_fscore_support(pixel_label, pixel_pred, average=None, labels=[1,2,3,4])
    macro_f1 = f1.mean()

    return p, r, f1, macro_f1

In [7]:
seg_nd = ['3d','4d']
seg_model = ['gmm', 'k-means']
cluster_num = [16,32,64,128]
mode = [PRECISION_SOLO, FSCORE_SOLO, PRECISION_50]
threshold = 0.5

header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']

for model in seg_model:
    for nd in seg_nd:
        for c in cluster_num:
            for m in mode:
                print('{}_{}_{}_label_mode_{}.csv'.format(model, nd, c, m))
                f = open('{}_{}_{}_label_mode_{}.csv'.format(model, nd, c, m), 'w')
                writer = csv.writer(f)
                writer.writerow(header)

                class_label = auto_label_last_201(model, nd, c, m, threshold)

                for s in range(400, 801):
                    p, r, f1, macro_f1_4 = evaluate_label_per_slice(model, nd, c, s, class_label)
                    f1_3 = np.array([f1[0], f1[1], f1[3]])
                    macro_f1_3 = f1_3.mean()
                    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
                    writer.writerow(data)

                f.close()
                print('Finish!')

gmm_3d_16_label_mode_4.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Finish!
gmm_3d_16_label_mode_3.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Finish!
gmm_3d_16_label_mode_1.csv


...

Finish!
k-means_4d_128_label_mode_3.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Finish!
k-means_4d_128_label_mode_1.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Finish!


In [8]:
seg_nd = ['3d','4d']
seg_model = ['gmm', 'k-means']
cluster_num = [16,32,64,128]
mode = PRECISION_X
threshold = [0.6,0.7,0.8,0.9]

header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']

for model in seg_model:
    for nd in seg_nd:
        for c in cluster_num:
            for t in threshold:
                print('{}_{}_{}_label_mode_{}_{}.csv'.format(model, nd, c, mode, t))
                f = open('{}_{}_{}_label_mode_{}_{}.csv'.format(model, nd, c, mode, t), 'w')
                writer = csv.writer(f)
                writer.writerow(header)

                class_label = auto_label_last_201(model, nd, c, mode, t)

                for s in range(400, 801):
                    p, r, f1, macro_f1_4 = evaluate_label_per_slice(model, nd, c, s, class_label)
                    f1_3 = np.array([f1[0], f1[1], f1[3]])
                    macro_f1_3 = f1_3.mean()
                    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
                    writer.writerow(data)

                f.close()
                print('Finish!')

gmm_3d_16_label_mode_2_0.6.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Finish!
gmm_3d_16_label_mode_2_0.7.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Finish!
gmm_3d_16_label_mode_2_0.8.csv


...

Finish!
k-means_4d_128_label_mode_2_0.8.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Finish!
k-means_4d_128_label_mode_2_0.9.csv


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Finish!


# Evaluation of Unsupervised Part

This version is updated in terms of the image dimension in large_cluster_rec, which is (700,855,3).

This part should not use macro or micro, for larger clusters, like 16,32,64,128. Because each cluster is not a fully predicted class yet. It is meaningless at this stage to consider macro score. Just get pre

However, in the following cell when using precison_recall_fscore_support, label and pred were not flatten, which means they are in format of 2d array. Each row was considered as a label as a whole, leading sklearn to detect as multi-class scenario. Also, we should not use macro and micro average for binary classification even if the data set is imbalanced, see (https://datascience.stackexchange.com/questions/85182/macro-and-micro-average-for-imbalanced-binary-classes#:~:text=In%20general%20micro-%20and%20macro-average%20performance%20are%20not,for%20the%20classifier%29%20and%20the%20majority%20class%20%28easier%29%3A) 

But at least the micro-averaged results actually yields the exact same results as using (average='binary', pos_label=True). Hence, we just use the results generated by this code to evaluate, rather than regenerating new spreadsheet.

In [ ]:
seg_nd = '3d'
seg_model = 'k-means'
cluster_num = 16

current_path = os.getcwd()
seg_path = os.path.join(current_path, 'large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))

header = ['slice', 'current_cluster', 'pore_micro_precision', 'pore_micro_recall', 'pore_micro_f1', 'pore_macro_precision', 'pore_macro_recall', 'pore_macro_f1', 
          'gypsum_micro_precision', 'gypsum_micro_recall', 'gypsum_micro_f1', 'gypsum_macro_precision', 'gypsum_macro_recall', 'gypsum_macro_f1', 
          'celestite_micro_precision', 'celestite_micro_recall', 'celestite_micro_f1', 'celestite_macro_precision', 'celestite_macro_recall', 'celestite_macro_f1', 
          'bassanite_micro_precision', 'bassanite_micro_recall', 'bassanite_micro_f1', 'bassanite_macro_precision', 'bassanite_macro_recall', 'bassanite_macro_f1']
f = open('{}_{}_{}_f1.csv'.format(seg_model, seg_nd, cluster_num), 'w')
writer = csv.writer(f)
writer.writerow(header)

for ref_slice_num in range(401):   # should be 401
    seg_res_list = glob.glob(os.path.join(seg_path, str(ref_slice_num+400), '*.png'))

    index_dragonfly = ''
    if (ref_slice_num+1) // 10 == 0:
        index_dragonfly = index_dragonfly + '00' + str(ref_slice_num+1)
    elif (ref_slice_num+1) // 100 == 0:
        index_dragonfly = index_dragonfly + '0' + str(ref_slice_num+1)
    else:
        index_dragonfly = str(ref_slice_num+1)
	
    path_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores'+index_dragonfly+'.tiff')
    path_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum'+index_dragonfly+'.tiff')
    path_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite'+index_dragonfly+'.tiff')
    path_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite'+index_dragonfly+'.tiff')

    # read reference images
    ref_pore = cv2.imread(path_pore)
    ref_gypsum = cv2.imread(path_gypsum)
    ref_celestite = cv2.imread(path_celestite)
    ref_bassanite = cv2.imread(path_bassanite)

    # transfer the reference into 0/1 labels
    pore_label = (ref_pore == [255,255,255]).all(axis=2)
    gypsum_label = (ref_gypsum == [255,255,255]).all(axis=2)
    celestite_label = (ref_celestite == [255,255,255]).all(axis=2)
    bassanite_label = (ref_bassanite == [255,255,255]).all(axis=2)

    for seg in seg_res_list:
        current_cluster = int(os.path.basename(seg[-16:-13]))

        # read segmented results
        img = cv2.imread(seg)
        img_crop = ~img

        # 0/1 labeling
        pred = (img_crop == [255,255,255]).all(axis=2)

        # metrics
        # pore
        pore_micro_precision, pore_micro_recall, pore_micro_f1, _ = precision_recall_fscore_support(pore_label, pred, average='micro')
        pore_macro_precision, pore_macro_recall, pore_macro_f1, _ = precision_recall_fscore_support(pore_label, pred, average='macro')
    
        # gypsum
        gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, _ = precision_recall_fscore_support(gypsum_label, pred, average='micro')
        gypsum_macro_precision, gypsum_macro_recall, gypsum_macro_f1, _ = precision_recall_fscore_support(gypsum_label, pred, average='macro')
        
        # celestite
        celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, _ = precision_recall_fscore_support(celestite_label, pred, average='micro')
        celestite_macro_precision, celestite_macro_recall, celestite_macro_f1, _ = precision_recall_fscore_support(celestite_label, pred, average='macro')

        # bassanite
        bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, _ = precision_recall_fscore_support(bassanite_label, pred, average='micro')
        bassanite_macro_precision, bassanite_macro_recall, bassanite_macro_f1, _ = precision_recall_fscore_support(bassanite_label, pred, average='macro')




        # write into csv file
        data = [ref_slice_num+400, current_cluster,pore_micro_precision, pore_micro_recall, pore_micro_f1, pore_macro_precision, pore_macro_recall, pore_macro_f1,
                gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, gypsum_macro_precision, gypsum_macro_recall, gypsum_macro_f1,
                celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, celestite_macro_precision, celestite_macro_recall, celestite_macro_f1,
                bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, bassanite_macro_precision, bassanite_macro_recall, bassanite_macro_f1]  # same structure as header
        writer.writerow(data)



f.close()


# Experiments and Notes

In [10]:
seg_nd = '3d'
seg_model = 'k-means'
cluster_num = 16

current_path = os.getcwd()
seg_path = os.path.join(current_path, 'large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))

# header = ['slice', 'current_cluster', 'pore_micro_precision', 'pore_micro_recall', 'pore_micro_f1', 'pore_macro_precision', 'pore_macro_recall', 'pore_macro_f1', 
#           'gypsum_micro_precision', 'gypsum_micro_recall', 'gypsum_micro_f1', 'gypsum_macro_precision', 'gypsum_macro_recall', 'gypsum_macro_f1', 
#           'celestite_micro_precision', 'celestite_micro_recall', 'celestite_micro_f1', 'celestite_macro_precision', 'celestite_macro_recall', 'celestite_macro_f1', 
#           'bassanite_micro_precision', 'bassanite_micro_recall', 'bassanite_micro_f1', 'bassanite_macro_precision', 'bassanite_macro_recall', 'bassanite_macro_f1']

# header = ['slice', 'current_cluster', 'pore_precision', 'pore_recall', 'pore_f1',
#           'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
#           'celestite_precision', 'celestite_recall', 'celestite_f1', 
#           'bassanite_precision', 'bassanite_recall', 'bassanite_f1']

header = ['a1', 'b1', 'c1', 'd1', 'a2', 'b2', 'c2', 'd2', 'a3', 'b3', 'c3', 'd3']

f = open('{}_{}_{}_f1_test.csv'.format(seg_model, seg_nd, cluster_num), 'w')
writer = csv.writer(f)
writer.writerow(header)

for ref_slice_num in range(1):   # should be 401
    seg_res_list = glob.glob(os.path.join(seg_path, str(ref_slice_num+400), '*.png'))

    index_dragonfly = ''
    if (ref_slice_num+1) // 10 == 0:
        index_dragonfly = index_dragonfly + '00' + str(ref_slice_num+1)
    elif (ref_slice_num+1) // 100 == 0:
        index_dragonfly = index_dragonfly + '0' + str(ref_slice_num+1)
    else:
        index_dragonfly = str(ref_slice_num+1)
	
    path_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores'+index_dragonfly+'.tiff')
    path_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum'+index_dragonfly+'.tiff')
    path_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite'+index_dragonfly+'.tiff')
    path_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite'+index_dragonfly+'.tiff')

    # read reference images
    ref_pore = cv2.imread(path_pore)
    ref_gypsym = cv2.imread(path_gypsum)
    ref_celestite = cv2.imread(path_celestite)
    ref_bassanite = cv2.imread(path_bassanite)

    # transfer the reference into 0/1 labels
    pore_label = (ref_pore == [255,255,255]).all(axis=2)
    gypsum_label = (ref_gypsym == [255,255,255]).all(axis=2)
    celestite_label = (ref_celestite == [255,255,255]).all(axis=2)
    bassanite_label = (ref_bassanite == [255,255,255]).all(axis=2)

    # concate label
    #label = pore_label + gypsum_label + celestite_label + bassanite_label   # newly added, need to compute the scores for multi-class; after assigning large number of clusters into 4, use to evaluate.

    for seg in seg_res_list:
        current_cluster = int(os.path.basename(seg[-16:-13]))

        # read segmented results
        img = cv2.imread(seg)
        img_crop = ~img

        # 0/1 labeling
        pred = (img_crop == [255,255,255]).all(axis=2)

        # # metrics
        # # pore
        # pore_micro_precision, pore_micro_recall, pore_micro_f1, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), average='micro')
        # pore_macro_precision, pore_macro_recall, pore_macro_f1, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), average='macro')
    
        # # gypsum
        # gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, _ = precision_recall_fscore_support(gypsum_label.flatten(), pred.flatten(), average='micro')
        # gypsum_macro_precision, gypsum_macro_recall, gypsum_macro_f1, _ = precision_recall_fscore_support(gypsum_label.flatten(), pred.flatten(), average='macro')
        
        # # celestite
        # celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, _ = precision_recall_fscore_support(celestite_label.flatten(), pred.flatten(), average='micro')
        # celestite_macro_precision, celestite_macro_recall, celestite_macro_f1, _ = precision_recall_fscore_support(celestite_label.flatten(), pred.flatten(), average='macro')

        # # bassanite
        # bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, _ = precision_recall_fscore_support(bassanite_label.flatten(), pred.flatten(), average='micro')
        # bassanite_macro_precision, bassanite_macro_recall, bassanite_macro_f1, _ = precision_recall_fscore_support(bassanite_label.flatten(), pred.flatten(), average='macro')

                
        a1, a2, a3, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), average='binary', pos_label=True)

        b1, b2, b3, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), average='micro')

        #a, b, c, _ = precision_recall_fscore_support(pore_label, pred, average='binary', pos_label=True)  """is not valid to use 'binary' because indeed count one row of pixel as an element of 1d array, thus are multi-classed, not binary"""

        c1, c2, c3, _ = precision_recall_fscore_support(pore_label, pred, average='macro')

        d1, d2, d3, _ = precision_recall_fscore_support(pore_label, pred, average='micro')

        """ a and d gave exact same results, so previous misunderstanding of the method did give correct result. Yet a is both computationally and conceptually correct, while d gives correct results but still not sure why."""

        # gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, _ = precision_recall_fscore_support(gypsum_label.flatten(), pred.flatten(), average='binary', pos_label=False)

        # celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, _ = precision_recall_fscore_support(celestite_label.flatten(), pred.flatten(), average='binary', pos_label=False)

        # bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, _ = precision_recall_fscore_support(bassanite_label.flatten(), pred.flatten(), average='binary', pos_label=False)

        #pore_micro_precision, pore_micro_recall, pore_micro_f1, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), labels=[True, False])

        # gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, _ = precision_recall_fscore_support(gypsum_label.flatten(), pred.flatten(), labels=[True, False])

        # celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, _ = precision_recall_fscore_support(celestite_label.flatten(), pred.flatten(), labels=[True, False])

        # bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, _ = precision_recall_fscore_support(bassanite_label.flatten(), pred.flatten(), labels=[True, False])

        data = [a1, b1,c1,d1,a2,b2,c2,d2, a3,b3,c3,d3]
        writer.writerow(data)





#         # write into csv file
#         data = [ref_slice_num+400, current_cluster,pore_micro_precision, pore_micro_recall, pore_micro_f1, pore_macro_precision, pore_macro_recall, pore_macro_f1,
#                 gypsum_micro_precision, gypsum_micro_recall, gypsum_micro_f1, gypsum_macro_precision, gypsum_macro_recall, gypsum_macro_f1,
#                 celestite_micro_precision, celestite_micro_recall, celestite_micro_f1, celestite_macro_precision, celestite_macro_recall, celestite_macro_f1,
#                 bassanite_micro_precision, bassanite_micro_recall, bassanite_micro_f1, bassanite_macro_precision, bassanite_macro_recall, bassanite_macro_f1]  # same structure as header
#         writer.writerow(data)



f.close()


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

# organiose

In [11]:
import os
import shutil
import glob

In [30]:


"""
This file help organise the segmentation results from the same slice into one folder
"""

project_path = os.getcwd()

top_folder_path = os.path.join(project_path, 'new_large_clusters_rec', 'gmm', '4d', 'cluster_3')
all_files = glob.glob(os.path.join(top_folder_path, '*.png'))


# make 401 new sub-dirs
# 400-801 for gmm
# # 391 - 811 for k-means
# for i in range (400, 801):
#     document_path_4D = os.path.join(top_folder_path, str(i))
#     os.mkdir(document_path_4D)


for seg in all_files:
    file_name = int(os.path.basename(seg)[10:14])
    dst = os.path.join(top_folder_path, str(file_name))
    shutil.move(seg, dst)


print('Finish organisation')

Finish organisation


# seg_res_evaluation.py

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import glob
from sklearn.metrics import precision_recall_fscore_support

3d-3: 10 mins
3d-4: 10 mins
3d-6: 20 min
3d-16: 1hr
3d-32: 106 mins
3d-64: 213 mins
3d-128: 419 mins




4d-3: 
4d-4: 
4d-6: 
4d-16: 
4d-32: 
4d-64: 
4d-128: 426 mins



In [11]:
seg_nd = '4d'
seg_model = 'gmm'
cluster_num = 6

current_path = os.getcwd()
seg_path = os.path.join(current_path, 'new_large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))

header = ['slice', 'current_cluster', 'pore_precision', 'pore_recall', 'pore_f1',  'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1']

eva_folder = os.path.join(os.getcwd(), 'seg_res_evaluation_csv')
if not os.path.exists(eva_folder):
    os.mkdir(eva_folder)

csv_path = os.path.join(eva_folder, '{}_{}_{}.csv'.format(seg_model, seg_nd, cluster_num))

f = open(csv_path, 'w')
writer = csv.writer(f)
writer.writerow(header)

for ref_slice_num in range(401):   # should be 401
    seg_res_list = glob.glob(os.path.join(seg_path, str(ref_slice_num+400), '*.png'))

    index_dragonfly = ''
    if (ref_slice_num+1) // 10 == 0:
        index_dragonfly = index_dragonfly + '00' + str(ref_slice_num+1)
    elif (ref_slice_num+1) // 100 == 0:
        index_dragonfly = index_dragonfly + '0' + str(ref_slice_num+1)
    else:
        index_dragonfly = str(ref_slice_num+1)
	
    path_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores'+index_dragonfly+'.tiff')
    path_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum'+index_dragonfly+'.tiff')
    path_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite'+index_dragonfly+'.tiff')
    path_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite'+index_dragonfly+'.tiff')

    # read reference images
    ref_pore = cv2.imread(path_pore)
    ref_gypsym = cv2.imread(path_gypsum)
    ref_celestite = cv2.imread(path_celestite)
    ref_bassanite = cv2.imread(path_bassanite)

    # transfer the reference into 0/1 labels
    pore_label = (ref_pore == [255,255,255]).all(axis=2)
    gypsum_label = (ref_gypsym == [255,255,255]).all(axis=2)
    celestite_label = (ref_celestite == [255,255,255]).all(axis=2)
    bassanite_label = (ref_bassanite == [255,255,255]).all(axis=2)

    for seg in seg_res_list:
        current_cluster = int(os.path.basename(seg[-16:-13]))

        # read segmented results
        img = cv2.imread(seg)
        img_crop = ~img    #[353:1053, 282:1137]   # consider to remove the cropping part, because we already cropped images for CNNs

        # 0/1 labeling
        pred = (img_crop == [255,255,255]).all(axis=2)

        # metrics
        # pore
        pore_precision, pore_recall, pore_f1, _ = precision_recall_fscore_support(pore_label.flatten(), pred.flatten(), average='binary', pos_label=True)
        gypsum_precision, gypsum_recall, gypsum_f1, _ = precision_recall_fscore_support(gypsum_label.flatten(), pred.flatten(), average='binary', pos_label=True)
        celestite_precision, celestite_recall, celestite_f1, _ = precision_recall_fscore_support(celestite_label.flatten(), pred.flatten(), average='binary', pos_label=True)
        bassanite_precision, bassanite_recall, bassanite_f1, _ = precision_recall_fscore_support(bassanite_label.flatten(), pred.flatten(), average='binary', pos_label=True)


        # write into csv file
        data = [ref_slice_num+400, current_cluster, pore_precision, pore_recall, pore_f1, gypsum_precision, gypsum_recall, gypsum_f1, 
                celestite_precision, celestite_recall, celestite_f1, bassanite_precision, bassanite_recall, bassanite_f1]  # same structure as header
        writer.writerow(data)



f.close()

c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le